In [1]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return x*x

In [2]:
with Pool(processes=4) as pool:
    # print "[0, 1, 4,..., 81]"
    print(pool.map(f, range(10)))

    # print same numbers in arbitrary order
    for i in pool.imap_unordered(f, range(10)):
        print(i)

    # evaluate "f(20)" asynchronously
    res = pool.apply_async(f, (20,))      # runs in *only* one process
    print(res.get(timeout=1))             # prints "400"

    # evaluate "os.getpid()" asynchronously
    res = pool.apply_async(os.getpid, ()) # runs in *only* one process
    print(res.get(timeout=1))             # prints the PID of that process

    # launching multiple evaluations asynchronously *may* use more processes
    multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
    print([res.get(timeout=1) for res in multiple_results])

    # make a single worker sleep for 10 secs
    res = pool.apply_async(time.sleep, (10,))
    try:
        print(res.get(timeout=1))
    except TimeoutError:
        print("We lacked patience and got a multiprocessing.TimeoutError")

    print("For the moment, the pool remains available for more work")

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0
1
4
16
9
36
25
49
64
81
400
75721
[75724, 75722, 75723, 75721]
We lacked patience and got a multiprocessing.TimeoutError
For the moment, the pool remains available for more work


# Multi-Core Parallelism

http://people.duke.edu/~ccc14/sta-663-2016/19B_Threads_Processses_Concurrency.html

In [7]:
%load_ext cython

In [8]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
from multiprocessing import Pool, Value, Array
import time
from numba import njit

In [9]:
import numpy as np

### Vanilla Python

In [10]:
def mc_pi(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [11]:
%%time
res = [mc_pi(int(1e7)) for i in range(10)]

KeyboardInterrupt: 

### using numba

In [13]:
@njit()
def mc_pi_numba(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [15]:
%%time

res = [mc_pi_numba(int(1e7)) for i in range(10)]

CPU times: user 1.2 s, sys: 4.04 ms, total: 1.2 s
Wall time: 1.2 s


### Using processes in parallel with ProcessPoolExecutor

In [32]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e7) for i in range(10)])

CPU times: user 12.1 ms, sys: 17.2 ms, total: 29.3 ms
Wall time: 413 ms


In [31]:
list(res)

[3.1420792,
 3.1413048,
 3.1418004,
 3.1412552,
 3.1412596,
 3.1407592,
 3.1424468,
 3.1414776,
 3.1405,
 3.1425184]

### Using processes in parallel with ThreadPoolExecutor

In [35]:
%%time

with ThreadPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e7) for i in range(10)])

CPU times: user 1.21 s, sys: 8.87 ms, total: 1.21 s
Wall time: 1.21 s


### Using multiprocessing

In [37]:
%%time

with mp.Pool(processes=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e7) for i in range(10)])

CPU times: user 11.6 ms, sys: 13.6 ms, total: 25.2 ms
Wall time: 434 ms
